In [4]:
import datasets
import pandas as pd
import transformers


In [5]:
gsm8k = datasets.load_dataset("gsm8k", "main")
t = transformers.AutoTokenizer.from_pretrained("HuggingFaceTB/SmolLM2-1.7B-Instruct")

In [23]:
def n_shot_text(*, gsm8k, tokenizer, N, use_cot, with_last_answer=False):
    turns = []

    for i, qa in enumerate(pd.DataFrame(gsm8k["train"][:N + 1]).to_dict(orient="records")):
        is_last = i == N + 1 - 1
        reasoning, answer = qa["answer"].split("####")
        turns.append({"role": "user", "content": qa["question"]})

        # In the default case, we don't want to add the last answer to the last turn,
        # because it's the model's job to generate the answer.
        # If with_last_answer is True, we override this behavior.
        if with_last_answer or not is_last:

            if use_cot:
                content = reasoning + " Answer: " + answer
            else:
                content = "Answer: " + answer

            turns.append({"role": "assistant", "content": content})

    return tokenizer.apply_chat_template(
        turns, 
        add_generation_prompt=not with_last_answer, 
        tokenize=False,
    )


In [24]:
print(n_shot_text(gsm8k=gsm8k, tokenizer=t, N=0, use_cot=False))

<|im_start|>system
You are a helpful AI assistant named SmolLM, trained by Hugging Face<|im_end|>
<|im_start|>user
Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?<|im_end|>
<|im_start|>assistant



In [21]:
print(n_shot_text(gsm8k, t, 2))

<|im_start|>system
You are a helpful AI assistant named SmolLM, trained by Hugging Face<|im_end|>
<|im_start|>user
Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?<|im_end|>
<|im_start|>assistant
Natalia sold 48/2 = <<48/2=24>>24 clips in May.
Natalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.
Answer:  72<|im_end|>
<|im_start|>user
Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?<|im_end|>
<|im_start|>assistant
Weng earns 12/60 = $<<12/60=0.2>>0.2 per minute.
Working 50 minutes, she earned 0.2 x 50 = $<<0.2*50=10>>10.
Answer:  10<|im_end|>
<|im_start|>user
Betty is saving money for a new wallet which costs $100. Betty has only half of the money she needs. Her parents decided to give her $15 for that purpose, and her grandparents twice as much as her parents. How much more money does Betty need to 